# Código prático do card 10
o intuíto é aproveitar o aprendizado e o código fornecido, adaptando para outra necessidade de negócio.
Criando um agente que tenha capacidade de gerar resultados dos jogos de futebol apenas de times do brasil, extraindo informações de um site que contém os resultados dos jogos de ontem do mundo todo.

In [1]:
# importação das bibliotecas necessárias para funcionamento do código

from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool
from crewai import Agent, Task, Crew
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# carregando a chave da openai par funcionamento do llm como serviço
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

D:\Users\paulo\PycharmProjects\fastcamp-agent\venv\lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
D:\Users\paulo\PycharmProjects\fastcamp-agent\venv\lib\site-packages\pydantic\_internal\_fields.py:198: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
D:\Users\paulo\PycharmProjects\fastcamp-agent\venv\lib\site-packages\pydantic\_internal\_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type w

In [15]:


# inicializando a ferramente de scrape e fornecendo um site que contém o resultado de jogos de hoje e de ontem

tool = ScrapeWebsiteTool(website_url='https://www.placardefutebol.com.br/jogos-de-ontem')  

# Extraindo o texto
text = tool.run()
# exibindo o texto
print(text)

# essa parte do código pode ser automatizada para extrair dados e persistir em uma base

Using Tool: Read website content

JOGOS DE ONTEM - Resultados dos jogos de ontem
Placar de Futebol
Ao vivo
Ontem
Jogos de hoje
Amanhã
Resultados dos jogos de ontem - 05/04/2025
Jogos de ontem, com o resultado das partidas das rodadas dos principais campeonatos do Brasil e do mundo. Saiba quem jogou e quem ganhou os jogos de ontem a tarde e ontem a noite.
Grupo de futebol no WhatsApp - Grupo de futebol no Telegram
MAIS POPULARES AGORA
Campeonato Espanhol
ENCERRADO
Sevilla
1
x
2
Atlético Madrid
Campeonato Inglês
INTERVALO
Manchester United
0
x
0
Manchester City
Campeonato Inglês
ENCERRADO
Fulham
3
x
2
Liverpool
Campeonato Alemão - Bundesliga
INTERVALO
Union Berlin
0
x
0
Wolfsburg
Campeonato Italiano
18 MIN
Atalanta
0
x
0
Lazio
Campeonato Brasileiro
ENCERRADO
Corinthians
3
x
0
Vasco
ENCERRADO
Ceará
2
x
0
Grêmio
ENCERRADO
Botafogo
2
x
0
Juventude
Ver tabela e classificação
Campeonato Brasileiro - Série B
ENCERRADO
Coritiba
1
x
0
Vila Nova
ENCERRADO
CRB
1
x
0
Chapecoense
ENCERRADO
América-M

In [16]:
# ferramenta para escrever o texto em um arquivo e salvá-lo no disco
file_writer_tool = FileWriterTool()
text = text.encode("ascii", "ignore").decode()
# escrevendo no disco
result = file_writer_tool._run(filename='resultados.txt', content = text, overwrite="True")
# visualizando o resultado
print(result)

Content successfully written to resultados.txt


In [17]:
# lendo o arquivo no disco com a ferramenta
tool = TXTSearchTool(txt='resultados.txt')

D:\Users\paulo\PycharmProjects\fastcamp-agent\venv\lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.51s/it]


In [1]:
# fornecendo um contexto para a ferramenta
context = tool.run('Quais os resultados dos jogos dos times brasileiros?')

# criando um agente com regra de jornalista esportivo, cujo objetivo é ler os dados salvos no arquivo e gerar a resposta correta obedecendo a restrição imposta
data_analyst = Agent(
    role='Jornalista esportivo',
    goal=f'Com base no contexto fornecido, responda a pergunta - Quais os resultados dos jogos dos times Brasileiros? Contexto - {context}',
    backstory='Você é um Jornalista esportivo com experiência em futebol',
    verbose=True,
    allow_delegation=False,
    tools=[tool]
)

# tarefa a ser executada pelo agente
test_task = Task(
    description="analisar resultados de jogos, colocando o nome dos times vencedores em negrito",
    tools=[tool],
    agent=data_analyst,
    expected_output='Crie uma lista com os nomes dos times que jogaram e o placar dos jogos no formato markdown'
)

# criando time de agentes com apenas um agente
crew = Crew(
    agents=[data_analyst],
    tasks=[test_task]
)

# iniciando a execução dos agentes
output = crew.kickoff()


NameError: name 'tool' is not defined

In [25]:
# exibindo o resultado da execução
print(output)

1. **Corinthians** 3 x 0 Vasco
2. **Ceará** 2 x 0 Grêmio
3. **Botafogo** 2 x 0 Juventude
4. **Coritiba** 1 x 0 Vila Nova
5. **CRB** 1 x 0 Chapecoense
6. **América-MG** 1 x 0 Botafogo-SP
7. Moto Club 0 x 0 Sampaio Corrêa
8. Imperatriz 0 x 0 Maranhão
9. **Oratório** 4 x 1 Santana
10. **Rio Branco-ES** 2 x 0 Porto Vitória
11. Barcelona-RO 1 x 1 Porto Velho
12. União-TO 0 x 0 Araguaína
13. Vitória Sub-20 1 x 1 Estrela de Março Sub-20
14. Náutico Sub-20 0 x 3 **Chapecoense Sub-20**
15. Brusque Sub-20 1 x 3 **Barra Sub-20**
16. Avaí Sub-20 3 x 4 **Joinville Sub-20**
17. **Criciúma Sub-20** 3 x 2 Figueirense Sub-20
18. Marcílio Dias Sub-20 1 x 1 Concórdia Sub-20
19. **Maranguape** 3 x 0 Itapipoca
20. **Goiás Sub-20** 1 x 0 Trindade Sub-20
21. Itauçu Sub-20 1 x 4 **Vila Nova Sub-20**
22. **Cerrado Sub-20** 2 x 0 Royal Sub-20
23. **Azuriz Sub-20** 3 x 2 Paraná Sub-20
24. Primavera 1 x 2 **Capivariano**
25. Taquaritinga 0 x 3 **AEA**
26. São Caetano 1 x 1 Nacional-SP
27. Joseense 0 x 2 **Paulist

In [26]:
# exibindo o resultado da execução no formato de marcadores
from IPython.display import Markdown
Markdown(output.raw)

1. **Corinthians** 3 x 0 Vasco
2. **Ceará** 2 x 0 Grêmio
3. **Botafogo** 2 x 0 Juventude
4. **Coritiba** 1 x 0 Vila Nova
5. **CRB** 1 x 0 Chapecoense
6. **América-MG** 1 x 0 Botafogo-SP
7. Moto Club 0 x 0 Sampaio Corrêa
8. Imperatriz 0 x 0 Maranhão
9. **Oratório** 4 x 1 Santana
10. **Rio Branco-ES** 2 x 0 Porto Vitória
11. Barcelona-RO 1 x 1 Porto Velho
12. União-TO 0 x 0 Araguaína
13. Vitória Sub-20 1 x 1 Estrela de Março Sub-20
14. Náutico Sub-20 0 x 3 **Chapecoense Sub-20**
15. Brusque Sub-20 1 x 3 **Barra Sub-20**
16. Avaí Sub-20 3 x 4 **Joinville Sub-20**
17. **Criciúma Sub-20** 3 x 2 Figueirense Sub-20
18. Marcílio Dias Sub-20 1 x 1 Concórdia Sub-20
19. **Maranguape** 3 x 0 Itapipoca
20. **Goiás Sub-20** 1 x 0 Trindade Sub-20
21. Itauçu Sub-20 1 x 4 **Vila Nova Sub-20**
22. **Cerrado Sub-20** 2 x 0 Royal Sub-20
23. **Azuriz Sub-20** 3 x 2 Paraná Sub-20
24. Primavera 1 x 2 **Capivariano**
25. Taquaritinga 0 x 3 **AEA**
26. São Caetano 1 x 1 Nacional-SP
27. Joseense 0 x 2 **Paulista**
28. **Flamengo Feminino** 2 x 0 Vasco Feminino